In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import dataframe_image as dfi
from datetime import datetime
import sklearn.metrics as mtr
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Função que cria os Modelos

In [2]:
def cria_modelo(df_teste, x_teste, y_teste, x_treino, y_treino):

    #Treino do Modelo XGB e salvamento do tempo
    inicio_tempo_treino_xgb = datetime.now()
    modelo_xgb = xgb.XGBRegressor(gamma = 0.0,
    learning_rate = 0.01,
    max_depth = 4,
    min_child_weight = 8,
    n_estimators = 500)
    modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)])

    fim_tempo_treino_xgb = datetime.now()
    tempo_treino_xgb = fim_tempo_treino_xgb - inicio_tempo_treino_xgb
    tempo_treino_xgb = float(f"{str(tempo_treino_xgb.seconds)}.{str(tempo_treino_xgb.microseconds)}")

    #Treino do Modelo RF e salvamento do tempo
    inicio_tempo_treino_rf = datetime.now()
    modelo_rf = RandomForestRegressor(bootstrap = True,
    max_depth = 11,
    max_features = 'log2',
    min_samples_leaf = 7,
    min_samples_split = 6,
    n_estimators = 500)
    modelo_rf.fit(x_treino, y_treino)

    fim_tempo_treino_rf = datetime.now()
    tempo_treino_rf = fim_tempo_treino_rf - inicio_tempo_treino_rf
    tempo_treino_rf = float(f"{str(tempo_treino_rf.seconds)}.{str(tempo_treino_rf.microseconds)}")

    #Treino do Modelo LR e salvamento do tempo
    inicio_tempo_treino_lr = datetime.now()
    modelo_lr = LinearRegression()
    modelo_lr.fit(x_treino, y_treino)

    fim_tempo_treino_lr = datetime.now()
    tempo_treino_lr = fim_tempo_treino_lr - inicio_tempo_treino_lr
    tempo_treino_lr = float(f"{str(tempo_treino_lr.seconds)}.{str(tempo_treino_lr.microseconds)}")

    #Nomeando a coluna que receberá as previsões
    TesteNomeXGB = f"XGB - {TesteNome}"
    TesteNomeRF = f"RF - {TesteNome}"
    TesteNomeLR = f"LR - {TesteNome}"

    #Predição do XGB e salvamento do tempo
    inicio_tempo_previsao_xgb = datetime.now()
    df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
    fim_tempo_previsao_xgb = datetime.now()
    tempo_previsao_xgb = fim_tempo_previsao_xgb - inicio_tempo_previsao_xgb
    tempo_previsao_xgb = float(f"{str(tempo_previsao_xgb.seconds)}.{str(tempo_previsao_xgb.microseconds)}")

    #Predição do RF e salvamento do tempo
    inicio_tempo_previsao_rf = datetime.now()
    df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
    fim_tempo_previsao_rf = datetime.now()
    tempo_previsao_rf = fim_tempo_previsao_rf - inicio_tempo_previsao_rf
    tempo_previsao_rf = float(f"{str(tempo_previsao_rf.seconds)}.{str(tempo_previsao_rf.microseconds)}")

    #Predição do LR e salvamento do tempo
    inicio_tempo_previsao_lr = datetime.now()
    df_teste[f'{TesteNomeLR}'] = modelo_lr.predict(x_teste)
    fim_tempo_previsao_lr = datetime.now()
    tempo_previsao_lr = fim_tempo_previsao_lr - inicio_tempo_previsao_lr
    tempo_previsao_lr = float(f"{str(tempo_previsao_lr.seconds)}.{str(tempo_previsao_lr.microseconds)}")

    #Métricas do XGB
    RRSE_xgb = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeXGB}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_xgb = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeXGB}"])

    #Métricas do RF
    RRSE_rf = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeRF}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_rf = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeRF}"])

    #Métricas do LR
    RRSE_lr = np.sqrt(sum((df_teste["tempo_viagem"] - df_teste[f"{TesteNomeLR}"]) ** 2) / sum((df_teste["tempo_viagem"] - np.mean(df_teste["tempo_viagem"])) ** 2))
    MAPE_lr = mtr.mean_absolute_percentage_error(df_teste["tempo_viagem"], df_teste[f"{TesteNomeLR}"])

    #Salvando as métricas num DataFrame Bonitinho
    metrics.append([f'{TesteNome}', Rota,'XGBoost','RRSE',RRSE_xgb, tempo_treino_xgb, tempo_previsao_xgb])
    metrics.append([f'{TesteNome}', Rota,'XGBoost','MAPE',MAPE_xgb, tempo_treino_xgb, tempo_previsao_xgb])
    metrics.append([f'{TesteNome}', Rota,'Random Forest','RRSE',RRSE_rf, tempo_treino_rf, tempo_previsao_rf])
    metrics.append([f'{TesteNome}', Rota,'Random Forest','MAPE',MAPE_rf, tempo_treino_rf, tempo_previsao_rf])
    metrics.append([f'{TesteNome}', Rota,'Linear Regression','RRSE',RRSE_lr, tempo_treino_lr, tempo_previsao_lr])
    metrics.append([f'{TesteNome}', Rota,'Linear Regression','MAPE',MAPE_lr, tempo_treino_lr, tempo_previsao_lr])

    #Salvando as predições num DataFrame Bonitinho
    for i in range(df_teste.shape[0]):
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'XGBoost',df_teste[f"{TesteNomeXGB}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Random Forest',df_teste[f"{TesteNomeRF}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Linear Regression',df_teste[f"{TesteNomeLR}"].iloc[i]])
        predictions.append([f'{TesteNome}', df_teste['data_partida'].iloc[i], Rota,'Tempo real',df_teste["tempo_viagem"].iloc[i]])

    # fi_xgb = pd.DataFrame(data=modelo_xgb.feature_importances_, index=modelo_xgb.feature_names_in_, columns=["importance"])
    # fi_xgb.sort_values("importance").plot(kind="barh", title="Importância Dados XGB")
    # plt.savefig(f"Imagens/Feature Importance/FI_XGB_{TesteNome}.jpg")

    # fi_rf = pd.DataFrame(data=modelo_rf.feature_importances_, index=x_treino.columns, columns=["importance"])
    # fi_rf.sort_values("importance").plot(kind="barh", title="Importância Dados RF")
    # plt.savefig(f"Imagens/Feature Importance/FI_RF_{TesteNome}.jpg")

    # coefs_lr = pd.DataFrame(modelo_lr.coef_, columns=['Coeficiente'], index = x_treino.columns)
    # coefs_lr.Coeficiente = coefs_lr.Coeficiente.abs()
    # coefs_lr['DesvioPadrão'] = x_treino.std(axis=0)
    # coefs_lr['Importancia'] = (100*coefs_lr['DesvioPadrão']*coefs_lr['Coeficiente'])/coefs_lr['DesvioPadrão']*coefs_lr['Coeficiente'].max()
    # coefs_lr['Importancia'].plot(kind='barh', title = 'Importância Dados LR')
    # plt.savefig(f"Imagens/Feature Importance/FI_LR_{TesteNome}.jpg")
    

# Parte pra editar os teste

In [22]:
df = pd.read_csv('Clean/rota135FL_Full.csv')
df.head()
Rota = "FL - 135"

In [21]:
def outlier_metric(valores):
    Q1 = np.percentile(valores,25)
    Q3 = np.percentile(valores,75)
    intervalo = Q3-Q1
    limite_superior = Q3+intervalo*1.5
    limite_inferior = Q1-intervalo*1.5
    return limite_superior,limite_inferior

# limite_superior,limite_inferior = outlier_metric(df['tempo_viagem'])
# df = df.loc[(df['tempo_viagem'] < limite_superior) & (df['tempo_viagem'] > limite_inferior)]

In [23]:
predictions = []
metrics = []

In [25]:
metrics = pd.DataFrame(metrics, columns=['Teste','Rota','Modelo','Métrica','Valor', 'Tempo de treino', 'Tempo de previsão'])
predictions = pd.DataFrame(predictions, columns=['Teste','Data','Rota','Modelo','Valor'])
# metrics.to_csv('metrics.csv', index = False)
# predictions.to_csv('predictions.csv', index = False)

In [26]:
metrics

,Teste,Rota,Modelo,Métrica,Valor,Tempo de treino,Tempo de previsão
0,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 135,XGBoost,RRSE,0.646712,1.685458,0.60020
1,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 135,XGBoost,MAPE,0.114051,1.685458,0.60020
2,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 135,Random Forest,RRSE,0.660434,2.141134,0.86195
3,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 135,Random Forest,MAPE,0.117582,2.141134,0.86195
4,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 135,Linear Regression,RRSE,0.859387,0.200400,0.99700
5,"0 Teste de Colunas - [dia_semana, tipo_dia, ho...",FL - 135,Linear Regression,MAPE,0.182404,0.200400,0.99700


In [6]:
Colunas = [
    #Primeira parte teste de colunas
    ["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "hora_dia"],
    ["dia_semana", "tipo_dia", "turno_dia"],
    ["dia_semana", "tipo_dia", "hora_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "turno_dia", "qtdDiasAno"],
    ["dia_semana", "tipo_dia", "hora_dia","partidaTimeStamp"],
    ["dia_semana", "tipo_dia", "turno_dia","partidaTimeStamp"],

    #Segunda parte teste com dados de clima
    ["dia_semana", "tipo_dia", "hora_dia", "Precipitacao", "Temperatura"],
    ["dia_semana", "tipo_dia", "hora_dia", "Chuva", "Calor"],
    ["dia_semana", "tipo_dia", "hora_dia", "Precipitacao"],
    ["dia_semana", "tipo_dia", "hora_dia", "Temperatura"],
    ["dia_semana", "tipo_dia", "hora_dia", "Chuva"],
    ["dia_semana", "tipo_dia", "hora_dia", "Calor"],

    #Terceira parte teste com os tempo de viagens anteriores
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2", "tempo_viagem_3", "tempo_viagem_4", "tempo_viagem_5"],

    #Quarta parte alternar percentual de treino
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"],
    ["dia_semana", "tipo_dia", "hora_dia", "tempo_viagem_1", "tempo_viagem_2"]
    
]

TestesNome = [
    #Primeira parte teste de colunas
    "0 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, turno_dia, qtdDiasAno, partidaTimeStamp]",
    # "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia]",
    # "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia]",
    # "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, qtdDiasAno]",
    # "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia, qtdDiasAno]",
    # "1 Teste de Colunas - [dia_semana, tipo_dia, hora_dia, partidaTimeStamp]",
    # "1 Teste de Colunas - [dia_semana, tipo_dia, turno_dia, partidaTimeStamp]",

    # #Segunda parte teste com dados de clima
    # "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Precipitacao, Temperatura]",
    # "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Chuva, Calor]",
    # "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Precipitacao]",
    # "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Temperatura]",
    # "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Chuva]",
    # "2 Teste Clima - [dia_semana, tipo_dia, hora_dia, Calor]",

    # #Terceira parte teste com os tempo de viagens anteriores
    # "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1]",
    # "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    # "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3]",
    # "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3, tempo_viagem_4]",
    # "3 Teste com os tempos de viagem anteriores - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2, tempo_viagem_3, tempo_viagem_4, tempo_viagem_5]",

    # #Quarta parte alternar percentual de treino
    # "4 Teste de quantidade de dados para treino - 2 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    # "4 Teste de quantidade de dados para treino - 4 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    # "4 Teste de quantidade de dados para treino - 6 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    # "4 Teste de quantidade de dados para treino - 8 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]",
    # "4 Teste de quantidade de dados para treino - 10 semanas - [dia_semana, tipo_dia, hora_dia, Precipitacao, tempo_viagem_1, tempo_viagem_2]"

]

PercentTeste = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.152173, 0.304347, 0.456521, 0.608695, 0.760869]


In [7]:
print(f'{len(Colunas)} --- {len(TestesNome)} --- {len(PercentTeste)}')

23 --- 1 --- 23


In [24]:
#for i in range(9,10,1):
for i in range(len(TestesNome)):
    shape = int(df.shape[0] * (1-PercentTeste[i]))
    df = df.sort_values(by=["data_partida"], ascending=False)
    df_teste = df[:shape]
    df_treino = df[shape:]

    TesteNome = TestesNome[i]

    x_teste = df_teste[Colunas[i]]
    y_teste = df_teste["tempo_viagem"]
    x_treino = df_treino[Colunas[i]]
    y_treino = df_treino["tempo_viagem"]

    cria_modelo(df_teste, x_teste, y_teste, x_treino, y_treino)

[0]	validation_0-rmse:3465.28773
[1]	validation_0-rmse:3431.56548
[2]	validation_0-rmse:3398.18880
[3]	validation_0-rmse:3365.15450
[4]	validation_0-rmse:3332.45917
[5]	validation_0-rmse:3300.09937
[6]	validation_0-rmse:3268.07195


[7]	validation_0-rmse:3236.37373
[8]	validation_0-rmse:3205.00130
[9]	validation_0-rmse:3173.95175
[10]	validation_0-rmse:3143.22184
[11]	validation_0-rmse:3112.80847
[12]	validation_0-rmse:3082.70803
[13]	validation_0-rmse:3052.91664
[14]	validation_0-rmse:3023.43264
[15]	validation_0-rmse:2994.25163
[16]	validation_0-rmse:2965.37208
[17]	validation_0-rmse:2936.78959
[18]	validation_0-rmse:2908.50240
[19]	validation_0-rmse:2880.48842
[20]	validation_0-rmse:2852.72513
[21]	validation_0-rmse:2825.28735
[22]	validation_0-rmse:2798.09457
[23]	validation_0-rmse:2771.18373
[24]	validation_0-rmse:2744.55142
[25]	validation_0-rmse:2718.23183
[26]	validation_0-rmse:2692.14860
[27]	validation_0-rmse:2666.33590
[28]	validation_0-rmse:2640.79174
[29]	validation_0-rmse:2615.54820
[30]	validation_0-rmse:2590.53161
[31]	validation_0-rmse:2565.77556
[32]	validation_0-rmse:2541.27771
[33]	validation_0-rmse:2517.06902
[34]	validation_0-rmse:2493.07878
[35]	validation_0-rmse:2469.33946
[36]	validation_0

C:\Users\leovi\AppData\Local\Temp\ipykernel_25008\1292773635.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeXGB}'] = modelo_xgb.predict(x_teste)
C:\Users\leovi\AppData\Local\Temp\ipykernel_25008\1292773635.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste[f'{TesteNomeRF}'] = modelo_rf.predict(x_teste)
C:\Users\leovi\AppData\Local\Temp\ipykernel_25008\1292773635.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

# Melhor Métrica

In [ ]:
df = pd.read_csv('metrics.csv')
df['Valor'] = df['Valor'].apply(lambda x: float(x))

In [ ]:
min_a = df.loc[df['Métrica'] == 'RRSE','Valor'].min()
a = df.loc[df['Valor'] == min_a]
a
#a.to_csv('melhor_rrse.csv',index=False)

In [ ]:
min_b = df.loc[df['Métrica'] == 'MAPE','Valor'].min()
b = df.loc[df['Valor'] == min_b]
b
#b.to_csv('melhor_mape.csv',index=False)